<a href="https://colab.research.google.com/github/viswambhar-yasa/AuToDiFf/blob/main/Solving_Burgers_Equation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Solve time discretized version of Burgers Equation ( suitable for analysis in various fields like sedimentation of polydispersive suspensions and colloids, aspect of turbulence, non-linear wave propagation, growth of molecular interfaces, longitudinal elastic waves in isotropic solids, traffic flow, cosmology, gas dynamics and shock wave theory.).

Data from: https://github.com/maziarraissi/PINNs

strategy:

->Output of NN is calculated simultaneously for all data points and loss is squared average from all data points.

->The loss is of both domain and boundary together. 

Type:

->Soft assignment of BCs. (explicitly not stated , but trained using data points)

->Since we are using time discretized version , the discretized version(formulated using Runge-Kutta Stepping scheme and solution at the beginning) is solved by the NN.


In [1]:
from os.path import join
from google.colab import drive

ROOT = "/content/drive"
drive.mount(ROOT)

!git clone https://github.com/viswambhar-yasa/AuToDiFf "/content/drive/My Drive/AutoDiff"

Mounted at /content/drive
fatal: destination path '/content/drive/My Drive/AutoDiff' already exists and is not an empty directory.


In [2]:
ls

drive/  sample_data/


In [3]:
import sys,os
sys.path.append('/content/drive/MyDrive/AutoDiff/')

In [4]:
ls

drive/  sample_data/


In [5]:
cd /content/drive/MyDrive/AutoDiff/

/content/drive/MyDrive/AutoDiff


In [6]:
#Import required packages
import autodiff as ad 
import numpy as np 
import matplotlib.pyplot as plt 
from NN_architecture import NeuralNetLSTM,xavier
from scipy.io import loadmat
from optimizers import Adam
import sys

In [7]:

#Since the setup is complex, recursionlimit should be set to a large value manually
sys.setrecursionlimit(10000)
def loss_calculator(model,x0,u0,IRK_weights,):
    """
    calculates squared loss at all data points in both domain and boundary .
    Inputs:
    model: The model to be trained 
    x0   : Data points 
    u0   : Solution at 0.1 second 
    IRK Weights : Butcher Tableau of corresponding Runge-Kutta weight matrix
    returns: mean squared loss
    """
    #converting data points into suitable type
    X=ad.Variable(x0,name="X")
    U = model.output(X)
    U1 = U[:,:-1]
    #Instantiating dummy variables to get forward gradients
    dummy1 = ad.Variable(np.ones((100,32)),name="dummy1")
    dummy2 =ad.Variable(np.ones((100,33)),name="dummy2")
    dummy3= np.ones((100,32))
    #g = ad.grad(U,[X],previous_grad=dummy2)[0]
    #print("g:",g().shape)
    gx = ad.grad(ad.grad(U,[X],dummy2)[0],[dummy2])[0]
    ux = gx[:,:-1]
    #print("ux",ux().shape)
    #g1 = ad.grad(g,[X],previous_grad=dummy1)[0]
    #print("g1",g1().shape)
    gxx= ad.grad(ad.grad(gx,[X],dummy2)[0],[dummy2])[0]
    uxx = gxx[:,:-1]
    
    #print("uxx",uxx().shape)
    #F = -U1*g + (0.01/np.pi)*g1
    F = -U1*ux + ((0.01/np.pi)*uxx)
    #Formulate the loss
    temp =0.4*ad.MatMul(F,IRK_weights.transpose())
    U0 = U - temp
    u0 = ad.Variable(u0,name="u0")
    #val = ad.ReduceSumToShape(U0,(250,1))
    #squared Sum over all axes
    lossd = ad.ReduceSumToShape(ad.Pow(U0 - u0,2),(1,1))
    #print(lossd())
    X1 = ad.Variable(np.vstack((-1.0,+1.0)),name="X1")
    Ub = model.output(X1)
    #Loss at boundary squared sum over all axes
    lossb = ad.ReduceSumToShape(ad.Pow(Ub,2),(1,1))
    loss = lossd + lossb
    return loss


In [20]:
!wget https://github.com/viswambhar-yasa/AuToDiFf/blob/main/Dataset/initial_data.mat './content/'
!wget https://github.com/viswambhar-yasa/AuToDiFf/blob/main/Dataset/irk32.txt '/content/'

--2021-09-15 13:07:29--  https://github.com/viswambhar-yasa/AuToDiFf/blob/main/Dataset/initial_data.mat
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘initial_data.mat’

initial_data.mat        [ <=>                ] 127.68K  --.-KB/s    in 0.08s   

2021-09-15 13:07:29 (1.62 MB/s) - ‘initial_data.mat’ saved [130741]

--2021-09-15 13:07:29--  http://./content/
Resolving . (.)... failed: No address associated with hostname.
wget: unable to resolve host address ‘.’
FINISHED --2021-09-15 13:07:29--
Total wall clock time: 0.4s
Downloaded: 1 files, 128K in 0.08s (1.62 MB/s)
--2021-09-15 13:07:29--  https://github.com/viswambhar-yasa/AuToDiFf/blob/main/Dataset/irk32.txt
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 200

In [21]:

#Loading of exact data for plotting and training
N = 100
data = loadmat('/content/initial_data.mat')
Exact = np.real(data['usol']).T 
idx_x = np.random.choice(Exact.shape[1], N, replace=False) 
t = data['t'].flatten()[:,None] 

    
idx_t0 = 10
u0 = Exact[idx_t0:idx_t0+1,idx_x].T
x = data['x'].flatten()[:,None] 

x0 = x[idx_x,:]
#Instantiating the NN
model = NeuralNetLSTM(20,1,1,33)
model.set_weights([xavier(i().shape[0],i().shape[1]) for i in model.get_weights()])
tmp = np.float32(np.loadtxt('/content/irk32.txt' , ndmin = 2))
IRK_weights = np.reshape(tmp[0:32**2+32], (32+1,32))
IRK_times = tmp[32**2+32:]
epochs = 3001
#Instantiating the optimizer
optimizer = Adam(len(model.get_weights()))

In [ ]:

#------------------------------------------------------Start of training-------------------------------------
for i in range(epochs):
    loss = loss_calculator(model,x0,u0,IRK_weights)
    #Exit condition
    if loss() <= 1:
        break
    if i % 50 ==0:
        print("Iteration",i)
        print("loss  epoch",loss())
    params = model.get_weights()
    grad_params = ad.grad(loss,model.get_weights())
    #print("grad params shape",grad_params[6]()) 
    new_params = [0 for _ in params]
    new_params = optimizer([i() for i in params], [i() for i in grad_params])
    model.set_weights(new_params)

Iteration 0
loss  epoch [[2608.97499018]]
Iteration 50
loss  epoch [[1563.20148582]]
Iteration 100
loss  epoch [[1343.9948343]]
Iteration 150
loss  epoch [[1044.9799628]]
Iteration 200
loss  epoch [[810.79330315]]
Iteration 250
loss  epoch [[715.37314854]]
Iteration 300
loss  epoch [[637.4338244]]
Iteration 350
loss  epoch [[532.93030439]]
Iteration 400
loss  epoch [[422.58067411]]
Iteration 450
loss  epoch [[321.32678727]]
Iteration 500
loss  epoch [[234.08103589]]
Iteration 550
loss  epoch [[166.76041982]]
Iteration 600
loss  epoch [[124.41250542]]
Iteration 650
loss  epoch [[96.45584139]]
Iteration 700
loss  epoch [[79.29623417]]
Iteration 750
loss  epoch [[69.31879234]]
Iteration 800
loss  epoch [[63.32912132]]
Iteration 850
loss  epoch [[59.43774709]]
Iteration 900
loss  epoch [[56.6280015]]
Iteration 950
loss  epoch [[54.34556883]]
Iteration 1000
loss  epoch [[52.27465311]]
Iteration 1050
loss  epoch [[50.23392386]]
Iteration 1100
loss  epoch [[48.26071829]]
Iteration 1150
loss  

In [1]:

#-------------------------------------------plotting-----------------------    
def f(x):
    """
    plots starter at t=0s
    inputs
    x : input array
    return solution at 0s
    """
    return -np.sin(np.pi*x)
temp = np.linspace(-1,1,100)
plt.plot(temp,f(temp),label="Initial Condition at t=0s")
ue = Exact[50:50+1,idx_x].T
plt.scatter(x0,ue,marker="x",label="Exact Solution at t=0.5s")
X = ad.Variable(np.sort(x0),name="X")
Unew = model.output(X)
U=Unew[:,-1]()

plt.scatter(np.sort(x0),U,marker="+",label="Predicted by NN t=0.5s")
plt.xlabel("x")
plt.ylabel("y")
plt.title("Training a NN to solve Burgers Equation")
plt.legend()
plt.show()

NameError: ignored